In [4]:
import pandas
import numpy as np
from datetime import datetime, date, time, timezone, timedelta
from dateutil.parser import parse as parse_datetime
# help(np.argwhere)
import re

In [18]:
# dir(pandas)
# help(pandas.read_excel)

fname = '/Users/ericfultz/Library/Containers/com.apple.mail/Data/Library/Mail Downloads/0105F474-9668-4775-873F-4FAB7A4CB6B0/20240105_BRANCOL_FO9.xlsx'
fname = '/Users/ericfultz/Library/Containers/com.apple.mail/Data/Library/Mail Downloads/A5616F99-8752-4B2C-87B7-A9FC50EEB600/20240105_BRANCOL_FO10.xlsx'
fname = '/Users/ericfultz/Library/Containers/com.apple.mail/Data/Library/Mail Downloads/8D12F1C6-AC2F-496D-9EDE-1FEE7E5AA1F7/20240103_SAINT PATRICK_FO3.xlsx'
fname = '/Users/ericfultz/Library/Containers/com.apple.mail/Data/Library/Mail Downloads/501976A3-CD93-4373-A4E7-2D63737F2D19/20240103_SAINT PATRICK_FO5_FO6.xlsx'
fname = '/Users/ericfultz/Library/Containers/com.apple.mail/Data/Library/Mail Downloads/A8FB63E3-CA0C-4185-A942-D41FDA73C641/20240103_SAINT PATRICK_FO7.xlsx'

# boat = 'brancol'

boat = 'stpatrick'

all_sheets = pandas.read_excel(fname, sheet_name=None)

sheet_names = list(filter(lambda k: re.match('^FO \d+', k), all_sheets.keys()))

curr_sheet = all_sheets[sheet_names[1]]


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [6]:


def findcell(sheet, needle):
    for col_name in list(sheet.keys()):
        try:
            start_idx = sheet[col_name].to_list().index(needle)
            return (col_name, start_idx+1)
        except ValueError:
            pass
    return None

def set_haul_grid_fetch_one(curr_sheet, set_haul_title_cell, cell_str, cell_offset):
    cols_index = list(curr_sheet.keys())
    
    if cols_index.index(set_haul_title_cell[0]) + cell_offset[0] >= len(cols_index):
        raise ValueError(f'index offset {cell_offset[0]} out of bounds in sheet {cols_index}')
    
#     print(set_haul_title_cell)
#     print(cell_offset)
#     print(cols_index.index(set_haul_title_cell[0]))
#     print(cols_index[cols_index.index(set_haul_title_cell[0]) + cell_offset[0]])
#     print(curr_sheet[cols_index[cols_index.index(set_haul_title_cell[0]) + cell_offset[0]]])
#     print(curr_sheet[cols_index[cols_index.index(set_haul_title_cell[0]) + cell_offset[0]]][set_haul_title_cell[1] + cell_offset[1]])
                                
    if curr_sheet[cols_index[cols_index.index(set_haul_title_cell[0]) + cell_offset[0]]][set_haul_title_cell[1] + cell_offset[1]] != cell_str:
        raise ValueError(f"can't find {cell_str}")
    return curr_sheet[cols_index[cols_index.index(set_haul_title_cell[0]) + cell_offset[0]]][set_haul_title_cell[1] + cell_offset[1]+1]


def set_haul_grid_fetch_all(curr_sheet, set_haul_title_cell):
    start_date = set_haul_grid_fetch_one(curr_sheet, set_haul_title_cell, 'start date', (0, 0))
    try:
        start_time = set_haul_grid_fetch_one(curr_sheet, set_haul_title_cell, 'start time (UTC)', (1, 0))
    except:
        start_time = set_haul_grid_fetch_one(curr_sheet, set_haul_title_cell, 'start time', (1, 0))
    start_lat = set_haul_grid_fetch_one(curr_sheet, set_haul_title_cell, 'latitude', (2, 0))
    start_lon = set_haul_grid_fetch_one(curr_sheet, set_haul_title_cell, 'longitude', (3, 0))
    finish_date = set_haul_grid_fetch_one(curr_sheet, set_haul_title_cell, 'finish date', (0, 2))
    try:
        finish_time = set_haul_grid_fetch_one(curr_sheet, set_haul_title_cell, 'finish time (UTC)', (1, 2))
    except:
        try:
            finish_time = set_haul_grid_fetch_one(curr_sheet, set_haul_title_cell, 'finish time (UTC', (1, 2))
        except:
            finish_time = set_haul_grid_fetch_one(curr_sheet, set_haul_title_cell, 'finish time', (1, 2))
    finish_lat = set_haul_grid_fetch_one(curr_sheet, set_haul_title_cell, 'latitude', (2, 2))
    finish_lon = set_haul_grid_fetch_one(curr_sheet, set_haul_title_cell, 'longitude', (3, 2))

    return (start_date, start_time, start_lat, start_lon, finish_date, finish_time, finish_lat, finish_lon)


brancol 2024-01-05 2024-01-26 We don't have the exact date of return to port : the vessel has stopped the camera before arriving at port but the last video is the 25th at almost midnight. The time to finish the trip (they are close to the port), we guessed the finish date was the 26th.  Pierre GUILLO-LOHAN


In [19]:


# trip info


if list(all_sheets.keys())[0] != 'TRIP':
    raise ValueError('first sheet should be trip info')

trip_sheet = all_sheets['TRIP']

if 'Fishing trip' in trip_sheet.keys():
    trip_start_date_cell = ('Fishing trip', 0)
else:
    trip_start_date_cell = findcell(curr_sheet, 'Fishing trip')
    if not setting_cell:
        raise ValueError("no 'Fishing trip' block in sheet")
        

trip_notes = ''
try:
    trip_notes = set_haul_grid_fetch_one(trip_sheet, trip_start_date_cell, 'General notes ', (7,0)) 
except:
    pass

trip_start_date = set_haul_grid_fetch_one(trip_sheet, trip_start_date_cell, 'Start date', (0,1)).date()
trip_end_date = set_haul_grid_fetch_one(trip_sheet, trip_start_date_cell, 'Finish date', (2,1)).date()
trip_id = boat + "_" + str(trip_start_date)

trip_data = dict(
trip_id = trip_id,
trip_start_date = trip_start_date,
trip_end_date = trip_end_date,
trip_notes = trip_notes,
obsv_name = set_haul_grid_fetch_one(trip_sheet, trip_start_date_cell, "Observer's name", (0,4))
)
trip_data = {k:[v] for k,v in trip_data.items()}
trip_df = pandas.DataFrame(trip_data)
trip_df = trip_df.set_index('trip_id')
# print(trip_df)


# set/haul info

set_number = 0

sets_df = None
fish_df = None

for sheet_name in sheet_names:
    curr_sheet = all_sheets[sheet_name]
    
    # look for 'latitude' in this sheet. If found, it's probably a sheet with set/haul data
    fao_code_cell = findcell(curr_sheet, 'FAO code')
    if not fao_code_cell:
        continue
    catchcondition_cell = findcell(curr_sheet, 'catch condition')
    if not catchcondition_cell:
        continue
    discardreason_cell = findcell(curr_sheet, 'reason for discard')
    if not discardreason_cell:
        continue
    if discardreason_cell[1] != catchcondition_cell[1] or catchcondition_cell[1] != discardreason_cell[1]:
        raise ValueError(f"cannot find header row for fish data in sheet {sheet_name}")
        
    # find other metadata values

    cols_index = list(curr_sheet.keys())
    if 'Setting' in cols_index:
        setting_cell = ('Setting', 0)
    else:
        setting_cell = findcell(curr_sheet, 'Setting')
        if not setting_cell:
            raise ValueError("no 'Setting' block in sheet")

    (set_start_date, set_start_time, set_start_lat ,
        set_start_lon ,set_end_date, set_end_time,set_end_lat ,
        set_end_lon,) = set_haul_grid_fetch_all(curr_sheet, setting_cell)


    if 'Hauling' in cols_index:
        hauling_cell = ('Hauling', 0)
    else:
        hauling_cell = findcell(curr_sheet, 'Hauling')
        if not hauling_cell:
            raise ValueError("no 'Hauling' block in sheet")

    (haul_start_date, haul_start_time, haul_start_lat ,
        haul_start_lon ,haul_end_date, haul_end_time,haul_end_lat ,
        haul_end_lon,) = set_haul_grid_fetch_all(curr_sheet, hauling_cell)

    set_number += 1
    set_id = trip_id + "_set_"+str(set_number).zfill(2)
    set_row = dict(
        set_id=set_id,
        trip_id=trip_id,
        set_number=set_number,

        set_start_datetime = datetime.combine(set_start_date.date(), set_start_time).replace(tzinfo=timezone.utc),
        set_start_lat = set_start_lat ,
        set_start_lon = set_start_lon ,

        set_end_datetime = datetime.combine(set_end_date.date(), set_end_time).replace(tzinfo=timezone.utc),
        set_end_lat = set_end_lat ,
        set_end_lon = set_end_lon ,

        haul_start_datetime = datetime.combine(haul_start_date.date(), haul_start_time).replace(tzinfo=timezone.utc),
        haul_start_lat = haul_start_lat,
        haul_start_lon = haul_start_lon ,

        haul_end_datetime = datetime.combine(haul_end_date.date(), haul_end_time).replace(tzinfo=timezone.utc),
        haul_end_lat = haul_end_lat,
        haul_end_lon = haul_end_lon
    )
#         print({k:v for k,v in set_row.items()})
    set_row = {k:[v] for k,v in set_row.items()}

#         print(set_row)
    set_df = pandas.DataFrame(set_row)
    set_df = set_df.set_index('set_id')
#         print(set_df)
    if sets_df is None:
        sets_df = set_df
    else:
        sets_df = sets_df.append(set_df)
    
#     print(sheet_name, fao_code_cell[1])

    reimport_sheet = pandas.read_excel(fname, sheet_name=sheet_name, skiprows=fao_code_cell[1])
    reimport_sheet = reimport_sheet.loc[:, ~reimport_sheet.columns.str.contains('^Unnamed: ')]
    def replace_catch_hour(catch_hour):
        if type(catch_hour) == str:
#             print(catch_hour)
            catch_hour = parse_datetime(catch_hour).time()
        haul_datetime = set_row['haul_start_datetime'][0]
        catch_datetime = haul_datetime.replace(hour=catch_hour.hour, minute=catch_hour.minute)
        if haul_datetime - catch_datetime > timedelta(hours=2):
            # the catch_datetime is somehow smaller than the start of the haul
            # this is outside of the haul window
            # this is probably because the haul started just before midnight, and continued to the next day
            catch_datetime += timedelta(days=1)
            end_datetime = set_row['haul_end_datetime'][0]
            if catch_datetime - end_datetime > timedelta(hours=2):
                # adding a day didn't work, now it's outside of the haul window on the other side
                raise ValueError(f'catch time {catch_hour} cannot fit between haul times {haul_datetime} - {end_datetime}')
        return catch_datetime
        
    reimport_sheet['catch_datetime'] = reimport_sheet['hour'].map(replace_catch_hour)
    
#     print(reimport_sheet)
    
    reimport_sheet['set_id'] = set_id
    reimport_sheet.insert(0, 'set_id', reimport_sheet.pop('set_id'))
    reimport_sheet['fish_id'] = reimport_sheet['set_id'] + "_fish_" + pandas.Series(map(lambda i: str(i).zfill(3), reimport_sheet.index.values))
    reimport_sheet = reimport_sheet.set_index('fish_id')

    if fish_df is None:
        fish_df = reimport_sheet
    else:
        fish_df = fish_df.append(reimport_sheet)


# trip_df

# sets_df

# fish_df.index = pandas.Index(fish_df['fish_id'])

# fish_df.pop('fish_id')

fish_df



,set_id,FAO code,scientific name,hour,latitude,longitude,kind of catch,future,catch condition,fate condition,reason for discard,good pratices ETP,catch_datetime
fish_id,,,,,,,,,,,,,
stpatrick_2024-01-03_set_01_fish_000,stpatrick_2024-01-03_set_01,GEM,Gempylus serpens,08:59:00,5.607000,-84.989,non commercial species,discarded,alive not injured,alive not injured,NaN,NaN,2024-01-07 08:59:00+00:00
stpatrick_2024-01-03_set_01_fish_001,stpatrick_2024-01-03_set_01,FAL,Carcharhinus falciformis,11:46:00,5.671131,-85.010,target species,retained,alive not injured,NaN,NaN,NaN,2024-01-07 11:46:00+00:00
stpatrick_2024-01-03_set_01_fish_002,stpatrick_2024-01-03_set_01,DOL,Coryphaena hippurus,12:40:00,NaN,NaN,target species,retained,NaN,NaN,NaN,NaN,2024-01-07 12:40:00+00:00
stpatrick_2024-01-03_set_01_fish_003,stpatrick_2024-01-03_set_01,DOL,Coryphaena hippurus,12:48:00,5.758222,-85.011,target species,retained,alive not injured,NaN,NaN,NaN,2024-01-07 12:48:00+00:00
stpatrick_2024-01-03_set_01_fish_004,stpatrick_2024-01-03_set_01,DOL,Coryphaena hippurus,13:01:00,5.776000,-85.008,target species,retained,alive not injured,NaN,NaN,NaN,2024-01-07 13:01:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
stpatrick_2024-01-03_set_07_fish_061,stpatrick_2024-01-03_set_07,XXX,Indeterminated,10:41:00,8.191000,-84.016,unknown,discarded,unknown,unknown,"Escaped before being seen by the camera, idenf...",NaN,2024-01-24 10:41:00+00:00
stpatrick_2024-01-03_set_07_fish_062,stpatrick_2024-01-03_set_07,DOL,Coryphaena hippurus,10:44:00,8.191000,-84.015,target species,retained,alive not injured,NaN,NaN,NaN,2024-01-24 10:44:00+00:00
stpatrick_2024-01-03_set_07_fish_063,stpatrick_2024-01-03_set_07,DOL,Coryphaena hippurus,10:46:00,8.191000,-84.014,target species,retained,alive not injured,NaN,NaN,NaN,2024-01-24 10:46:00+00:00


In [20]:
import awswrangler as wr
import boto3
boto3.setup_default_session(profile_name='AWSAdministratorAccess-512267747959')

In [21]:
# dir(wr.s3)
# wr.s3.list_buckets()
bucket='51-gema-dev-dp-raw'
# wr.s3.list_directories(f's3://{bucket}/tnc_edge/')
# help(wr.s3.to_csv)


print(
    wr.s3.to_csv(trip_df, f's3://{bucket}/tnc_edge/{boat}_v1_bv_trips/{trip_id}.csv'),
    wr.s3.to_csv(sets_df, f's3://{bucket}/tnc_edge/{boat}_v1_bv_sets/{trip_id}.csv'),
    wr.s3.to_csv(fish_df, f's3://{bucket}/tnc_edge/{boat}_v1_bv_fish/{trip_id}.csv')
)


{'paths': ['s3://51-gema-dev-dp-raw/tnc_edge/stpatrick_v1_bv_trips/stpatrick_2024-01-03.csv'], 'partitions_values': {}} {'paths': ['s3://51-gema-dev-dp-raw/tnc_edge/stpatrick_v1_bv_sets/stpatrick_2024-01-03.csv'], 'partitions_values': {}} {'paths': ['s3://51-gema-dev-dp-raw/tnc_edge/stpatrick_v1_bv_fish/stpatrick_2024-01-03.csv'], 'partitions_values': {}}
